In [24]:
#Discovered geojson and county maps, going to create with that 
#First read in FIPS codes into database, then assign random value and do test plot
#Following https://plotly.com/python/choropleth-maps/ as a guide
!pip install plotly

     --------------------------------------- 15.6/15.6 MB 50.4 MB/s eta 0:00:00


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Python311\\etc'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip available: 22.3 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import mysql.connector
import requests
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup


C:\Users\jpehl\AppData\Local\Temp\ipykernel_3612\1444991554.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
url = "https://en.wikipedia.org/wiki/List_of_United_States_FIPS_codes_by_county"
req = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
soup = BeautifulSoup(req.text, 'html.parser')


In [8]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-client-prefs-pinned-disabled vector-toc-available" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of United States FIPS codes by county - Wikipedia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-

In [4]:
tables = soup.find_all('table')


C:\Users\jpehl\AppData\Local\Temp\ipykernel_3612\1333333814.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(str(tables[0]))


In [13]:
dfs = pd.read_html(str(tables[1]))

C:\Users\jpehl\AppData\Local\Temp\ipykernel_3612\1695778445.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(str(tables[1]))


In [16]:
print(dfs[0].columns)

Index(['FIPS', 'County or equivalent', 'State or equivalent'], dtype='object')


In [55]:
queries = dfs[0][['County or equivalent', 'State or equivalent']].copy()
queries['County or equivalent'] = queries['County or equivalent'].str.replace(' ', '+')

In [56]:
queries['query'] = queries['County or equivalent'] + "+" + queries['State or equivalent'] + "+coordinates"
print(queries['query'])

0          Autauga+County+Alabama+coordinates
1          Baldwin+County+Alabama+coordinates
2          Barbour+County+Alabama+coordinates
3             Bibb+County+Alabama+coordinates
4           Blount+County+Alabama+coordinates
                        ...                  
3238    Sweetwater+County+Wyoming+coordinates
3239         Teton+County+Wyoming+coordinates
3240         Uinta+County+Wyoming+coordinates
3241      Washakie+County+Wyoming+coordinates
3242        Weston+County+Wyoming+coordinates
Name: query, Length: 3243, dtype: object


In [61]:
from lxml import html
import time
for term in queries['query']:
    search_url = f'https://www.google.com/search?q={term}'
    print(search_url)
    req = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    time.sleep(1)
    soup = BeautifulSoup(req.text, 'html.parser')
    coords=  soup.find('div', class_='Z0LcW t2b5Cf')
    print(coords)


https://www.google.com/search?q=Autauga+County+Alabama+coordinates
None
https://www.google.com/search?q=Baldwin+County+Alabama+coordinates
None
https://www.google.com/search?q=Barbour+County+Alabama+coordinates


KeyboardInterrupt: 

  Using cached requests_html-0.10.0-py3-none-any.whl (13 kB)
  Using cached pyquery-2.0.0-py3-none-any.whl (22 kB)
  Using cached pyppeteer-1.0.2-py3-none-any.whl (83 kB)
  Using cached importlib_metadata-7.0.1-py3-none-any.whl (23 kB)
  Using cached cssselect-1.2.0-py2.py3-none-any.whl (18 kB)


ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'C:\\Python311\\Scripts\\pyppeteer-install.exe' -> 'C:\\Python311\\Scripts\\pyppeteer-install.exe.deleteme'


[notice] A new release of pip available: 22.3 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Requesting the temperature via google is too slow, will instead get latitude and longitude from google and use the weather.gov api to get the temperatures.


In [6]:
print(dfs[0])

    0                                                  1
0 NaN  This article may be too long to read and navig...
